In [ ]:
%matplotlib notebook

import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import pandas as pd
import seaborn as sns
from matplotlib.ticker import MaxNLocator


<b>Parameters (Replace f)</b>

In [ ]:
nu = 2
L = 10
E = 500
Ro = 5
mu = 96
delta = 1.5
b = 2
zeta = 1
U = 35
beta = 1/6*(np.sqrt(2)+np.log(1+np.sqrt(2)))
f = 0.85

#z = 2*beta/3
#print(z + np.sqrt(z**2 + z + 1))

In [ ]:
M = 10000
l_min = L/50
l_max = L/2
l = np.linspace(l_min,l_max,M)
#f = np.linspace(0,1,N) use f from simulations
Ci = np.zeros((M))
capacity = np.zeros((M))

<b> New York Probability distribution (Use the distribution you have used)</b>

In [ ]:
#print(beta)
k = 3.1
d_mean = 2.7

def nyc(x,d_mean):
    if (x==0):
        return 0
    else:
        return ((x/d_mean)**(-k))*np.exp(-d_mean/x)
    
normalize = scipy.integrate.quad(nyc,0,np.infty,args=(d_mean))[0]

def nycprob(x):
    return scipy.integrate.quad(nyc,0,x,args=(d_mean))[0]/normalize

$f = \frac{N_{bus}\left(\int_0^{\zeta\ell} rp(r)dr + \beta \ell \int_{\zeta \ell}^{\infty}p(r)dr\right)\nu E L^{2}}{U}$

where $N_{bus}$ is number of cabs/busses
U is the average speed

<b>For Tram Size<b>

In [ ]:
kappa = np.zeros((M))

def size(x,dx):
    return nyc(x,d_mean)*np.arccos(dx/x)/(2*np.pi*normalize)

def tramsize(ell):
    scl = int(L/ell)
#    print(scl)
    sm = 0
    for i in range(0,100):
        dx = ell*i
        sm = sm + 2*scipy.integrate.quad(size,np.maximum(i*ell,zeta*ell),np.infty,args=(dx))[0]
    sm = sm - (1-nycprob(zeta*ell))/4
    sm = sm*nu*E*ell*ell
    
    return sm
    #    return sm/mu

# for i in range(0,M):
#     cap = tramsize(l[i])
#     kappa[i] = cap

# print(kappa)

In [ ]:
def rpr(x):
    if x==0:
        return 0
    else:
        return x*nyc(x,d_mean)/normalize
    
den = nu*E*L*L*scipy.integrate.quad(rpr,0,np.infty)[0]

for i in range(0,M):
    dbus = ((scipy.integrate.quad(rpr,0,zeta*l[i]))[0] + beta*l[i]*(1-nycprob(zeta*l[i])))*nu*E*L*L
    #mu = tramsize(l[i])/100
    cap = tramsize(l[i])
    ctram = 2*mu*L*(L/l[i]+1)*Ro
    Ci[i] = (((delta/b)*f)*dbus + (1-f)*scipy.integrate.quad(rpr,0,np.infty)[0]*nu*E*L*L + ctram)/den
    capacity[i] = cap*f

<b> Capacity is for the tram capacity and Ci is the carbon index above. <b>